## Importing Libraries

In [1]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rXpsQeSvEgYtLPgpww6K1goOQm-78eTM_3UvUt1kGOwC',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'digitalnaturalist-donotdelete-pr-hzwfdprbfzwrlf'
object_key = 'Digital_Naturalist_Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
pwd

'/home/wsuser/work'

In [13]:
from io import BytesIO
import zipfile
unzip= zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
filepath =unzip.namelist()
for path in filepath:
    unzip.extract(path)

In [25]:
path

'Digital Naturalist Dataset/trainset/Mammal/Senenca White Deer Mammal/images (9).jpg'

## Downloading InceptionV3 pretrained model

In [16]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

## Freezing the weights in the pretrained model

In [17]:
for layer in base_model.layers:
  layer.trainable = False

## Flatening and adding dense layer


In [18]:
X = Flatten()(base_model.output)

#using sigmoid function as activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

## Compiling the model

In [19]:
model = Model(base_model.input,X)

#using adam as the optimizer
#accuracy is set as the metrics to judge the performance of the model
model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

#summarizing the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

## loading the train and test dataset

In [27]:
ls

'Digital Naturalist Dataset'/


In [28]:
filenames= os.listdir('/home/wsuser/work/Digital Naturalist Dataset/trainset')

In [39]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory='/home/wsuser/work/Digital Naturalist Dataset/trainset',target_size=(224,224),batch_size=100, color_mode="rgb",class_mode="input")

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory='/home/wsuser/work/Digital Naturalist Dataset/testset',target_size=(224,224),batch_size=100,color_mode="rgb",class_mode="input")

Found 114 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [30]:
#listing the found classes from train dataset
traindata.class_indices

{'Bird': 0, 'Flower': 1, 'Mammal': 2}

## Model Checkpoint and Early Stopping

In [40]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./model.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

In [41]:
traindata

# Model fitting

In [50]:
from keras.models import Model
from keras.layers import Input, Dense
encoding_dim = 32
input_img = Input(shape=(65536,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(65536, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')


In [ ]:
# Fitting the model 
modelHistory = model.fit(traindata, steps_per_epoch=60,epochs = 30,callbacks=call_back,validation_data=testdata)

## Exporting the model

In [ ]:
# Exporting the model to json 
model_json = model.to_json()
with open("DigitalNaturalist.json", "w") as json_file:
    json_file.write(model_json)

# Exporting the model weights

model.save_weights("DigitalNaturalist")
print("Saved model to disk")

Saved model to disk


# Testing the model

In [ ]:
#Testing the model 
predictions = ["Corpse Flower", 
               "Great Indian Bustard", 
               "Lady's slipper orchid", 
               "Pangolin", 
               "Spoon Billed Sandpiper", 
               "Seneca White Deer"
              ]


path = '/content/Test Data/GIB_AUG/aug_download (1)_0_2011.jpg'
ime = tf.keras.utils.load_img(path,target_size=(224,224))

i = tf.keras.preprocessing.image.img_to_array(ime)
i = preprocess_input(i)
input = np.array([i])
pred = model.predict(input)
predictions[np.argmax(pred)]

1/1 [==============================] - 0s 39ms/step


"Great Indian Bustard"

In [ ]:
model.save('DigitalNaturalist.h5')